In [1]:
matplotlib_style = 'fivethirtyeight'
import matplotlib.pyplot as plt; plt.style.use(matplotlib_style)
import numpy as np
import tensorflow as tf
import os
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sksurv.linear_model.coxph import BreslowEstimator
matplotlib_style = 'fivethirtyeight'
import matplotlib.pyplot as plt; plt.style.use(matplotlib_style)
from sklearn.model_selection import train_test_split
from utility.training import get_data_loader, scale_data_valid, make_time_event_split
from tools.model_builder import make_mcd_model
from utility.config import load_config
from utility.loss import CoxPHLoss
import paths as pt
from utility.survival import get_breslow_survival_times, compute_survival_times
from utility.risk import InputFunction
from tools.model_trainer import Trainer

import warnings
warnings.filterwarnings('ignore')

N_EPOCHS = 1
MLP_RUNS = 1
VI_RUNS = 100
MCD_RUNS = 100
DATASET_NAME = "SEER"
MODEL_VERSION = "FULL"

# Load config
config = load_config(pt.MLP_CONFIGS_DIR, f"{DATASET_NAME}.yaml")
optimizer = tf.keras.optimizers.deserialize(config['optimizer'])
loss_fn = CoxPHLoss()
activation_fn = config['activiation_fn']
layers = config['network_layers']
dropout_rate = config['dropout_rate']
l2_reg = config['l2_reg']
batch_size = config['batch_size']

# Load data
dl = get_data_loader(DATASET_NAME).load_data()
X, y = dl.get_data()
num_features, cat_features = dl.get_features()

# Split data in train, valid and test set
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.7)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5)

# Scale data
X_train, X_valid, X_test = scale_data_valid(X_train, X_valid, X_test, cat_features, num_features)
X_train = np.array(X_train)
X_valid = np.array(X_valid)
X_test = np.array(X_test)

# Make time/event split
t_train, e_train = make_time_event_split(y_train)
t_valid, e_valid = make_time_event_split(y_valid)
t_test, e_test = make_time_event_split(y_test)

# Make event times
lower, upper = np.percentile(t_test[t_test.dtype.names], [10, 90])
event_times = np.arange(lower, upper+1)

# Create model instance
mcd_model = make_mcd_model(input_shape=X_train.shape[1:], output_dim=2,
                           layers=layers, activation_fn=activation_fn,
                           dropout_rate=dropout_rate, regularization_pen=l2_reg)

# Make data loaders
train_ds = InputFunction(X_train, t_train, e_train, batch_size=batch_size, drop_last=True, shuffle=True)()
valid_ds = InputFunction(X_valid, t_valid, e_valid, batch_size=batch_size)() 
test_ds = InputFunction(X_test, t_test, e_test, batch_size=batch_size)()

# Make model
mcd_model = make_mcd_model(input_shape=X_train.shape[1:], output_dim=2,
                           layers=layers, activation_fn=activation_fn,
                           dropout_rate=dropout_rate, regularization_pen=l2_reg)

# Make trainer
trainer = Trainer(model=mcd_model, model_name="MCD",
                  train_dataset=train_ds, valid_dataset=None,
                  test_dataset=test_ds, optimizer=optimizer,
                  loss_function=loss_fn, num_epochs=N_EPOCHS,
                  event_times=event_times)

c:\Users\au475271\Miniconda3\envs\py38-fortuna\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Train
trainer.train_and_evaluate()
model = trainer.model

Completed MCD epoch 1/1
Model test variance: 0.8113531060665515


In [3]:
# Use only uncensored times
valid_idx = list(np.where(y_valid['event'] == True)[0])
X_valid = X_valid[valid_idx]
test_idx = list(np.where(y_test['event'] == True)[0])
X_test = X_test[test_idx]

In [7]:
n_samples = 100
valid_model_cpd = np.zeros((n_samples, len(X_valid)))
for i in range(0, n_samples):
    pred = np.reshape(model.predict(X_valid, verbose=0), len(X_valid))
    valid_model_cpd[i,:] = compute_survival_times(pred, t_train, e_train)

valid_means = np.mean(valid_model_cpd, axis=0)
valid_vars = np.var(valid_model_cpd, axis=0)
valid_stds = np.std(valid_model_cpd, axis=0)

test_model_cpd = np.zeros((n_samples, len(X_test)))
for i in range(0, n_samples):
    pred = np.reshape(model.predict(X_test, verbose=0), len(X_test))
    test_model_cpd[i,:] = compute_survival_times(pred, t_train, e_train)
test_means = np.mean(test_model_cpd, axis=0)
test_vars = np.var(test_model_cpd, axis=0)
test_stds = np.std(test_model_cpd, axis=0)

In [8]:
calib_outputs = np.concatenate((valid_means[:,np.newaxis], np.log(valid_vars)[:,np.newaxis]), axis=-1)
test_outputs = np.concatenate((test_means[:,np.newaxis], np.log(test_vars)[:,np.newaxis]), axis=-1)
calib_targets = y_valid[y_valid['event'] == True]['time'][:,np.newaxis]
test_targets = y_test[y_test['event'] == True]['time'][:,np.newaxis]
valid_means = valid_means[:,np.newaxis]
valid_stds = valid_stds[:,np.newaxis]

In [9]:
from fortuna.output_calib_model import OutputCalibRegressor, Config, Monitor

calib_model = OutputCalibRegressor()
calib_status = calib_model.calibrate(
    calib_outputs=calib_outputs,
    calib_targets=calib_targets,
    config=Config(monitor=Monitor(early_stopping_patience=2)),
)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
Epoch: 100 | loss: 34.17826: 100%|██████████| 100/100 [00:00<00:00, 269.23it/s]


In [21]:
calib_model

In [10]:
test_log_probs = calib_model.predictive.log_prob(
    outputs=test_outputs, targets=test_targets
)
test_means = calib_model.predictive.mean(outputs=test_outputs)
test_variances = calib_model.predictive.variance(outputs=test_outputs)
test_stds = calib_model.predictive.std(outputs=test_outputs, variances=test_variances)
test_cred_intervals = calib_model.predictive.credible_interval(outputs=test_outputs)

In [11]:
from fortuna.metric.regression import root_mean_squared_error
rmse = root_mean_squared_error(preds=test_means, targets=test_targets)
print(f"Test RMSE: {rmse}")

Test RMSE: 84.87664031982422


In [12]:
from fortuna.metric.regression import prediction_interval_coverage_probability

conformal_picp = prediction_interval_coverage_probability(
    lower_bounds=test_cred_intervals[:, 0],
    upper_bounds=test_cred_intervals[:, 1],
    targets=test_targets,
)
print(f"Calibrated PICP for 95% conformal intervals of test inputs: {conformal_picp}")

Calibrated PICP for 95% conformal intervals of test inputs: 0.7977527976036072


In [13]:
from fortuna.conformal import OneDimensionalUncertaintyConformalRegressor
test_cred_intervals = (
    OneDimensionalUncertaintyConformalRegressor().conformal_interval(
        val_preds=valid_means,
        val_uncertainties=valid_stds,
        test_preds=test_means,
        test_uncertainties=test_stds,
        val_targets=calib_targets,
        error=0.05,
    ) 
)
conformal_picp = prediction_interval_coverage_probability(
    lower_bounds=test_cred_intervals[:, 0],
    upper_bounds=test_cred_intervals[:, 1],
    targets=test_targets,
)
rmse = root_mean_squared_error(preds=test_means, targets=test_targets)
print(f"PICP for 95% conformal intervals of test inputs: {conformal_picp}")

PICP for 95% conformal intervals of test inputs: 0.9775280952453613


In [ ]:
# Use Brier score as loss function for calibration